In [80]:
#Install necessary packages (Run in Colab)
!pip install pandas numpy pathway bokeh --quiet

# Imports
import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
output_notebook()

# Load dataset
df = pd.read_csv('dataset.csv')

# Combine Date and Time into single datetime column
# Combine Date and Time into single datetime column (with dayfirst)
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df = df.sort_values('Timestamp')

# Normalize Traffic & VehicleType
traffic_mapping = {'low': 1, 'average': 2, 'high': 3}
vehicle_mapping = {'cycle': 1, 'bike': 2, 'car': 3, 'truck': 4}

df['Traffic'] = df['TrafficConditionNearby'].map(traffic_mapping)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_mapping)

# Pricing Models

BASE_PRICE = 10

def baseline_model(prev_price, occupancy, capacity, alpha=5):
    return prev_price + alpha * (occupancy / capacity)

def demand_model(row, base_price=BASE_PRICE, alpha=0.5, beta=0.3, gamma=0.2, delta=0.5, epsilon=0.2):
    demand = (
        alpha * (row['Occupancy'] / row['Capacity']) +
        beta * row['QueueLength'] -
        gamma * row['Traffic'] +
        delta * row['IsSpecialDay'] +
        epsilon * row['VehicleWeight']
    )
    demand = np.clip(demand, 0, 10)  # Normalize demand
    price = base_price * (1 + 0.1 * (demand / 10))
    price = np.clip(price, base_price * 0.5, base_price * 2)
    return price

def competitive_model(row, df_all, base_price=BASE_PRICE):
    competitors = df_all[
        (df_all['Latitude'] == row['Latitude']) &
        (df_all['Longitude'] == row['Longitude']) &
        (df_all['Timestamp'] == row['Timestamp'])
    ]
    competitor_prices = competitors['Price'].values
    avg_competitor_price = np.mean(competitor_prices) if len(competitor_prices) else base_price

    price = demand_model(row, base_price)
    # If full and competitors cheaper → suggest reducing price
    if row['Occupancy'] >= row['Capacity'] and price > avg_competitor_price:
        price = avg_competitor_price * 0.95
    # If competitors expensive → increase slightly
    elif avg_competitor_price > price:
        price *= 1.05
    return np.clip(price, base_price * 0.5, base_price * 2)

# Simulate Pricing over Dataset

df['Price'] = BASE_PRICE
prices_over_time = []

for idx, row in df.iterrows():
    if idx == 0:
        price = baseline_model(BASE_PRICE, row['Occupancy'], row['Capacity'])
    else:
        prev_price = df.iloc[idx - 1]['Price']
        price = baseline_model(prev_price, row['Occupancy'], row['Capacity'])
    df.at[idx, 'BaselinePrice'] = price

    # Demand-based Model
    df.at[idx, 'DemandPrice'] = demand_model(row)

    # Competitive Model
    df.at[idx, 'CompetitivePrice'] = competitive_model(row, df)

    # For Visualization
    prices_over_time.append({
        'Timestamp': row['Timestamp'],
        'BaselinePrice': df.at[idx, 'BaselinePrice'],
        'DemandPrice': df.at[idx, 'DemandPrice'],
        'CompetitivePrice': df.at[idx, 'CompetitivePrice']
    })

# Convert for Visualization
price_df = pd.DataFrame(prices_over_time)

# Real-Time Visualization using Bokeh
p = figure(title="Real-time Parking Price Simulation", x_axis_type='datetime', width=800, height=400)
p.line(price_df['Timestamp'], price_df['BaselinePrice'], color='blue', legend_label="Baseline Price")
p.line(price_df['Timestamp'], price_df['DemandPrice'], color='green', legend_label="Demand Price")
p.line(price_df['Timestamp'], price_df['CompetitivePrice'], color='red', legend_label="Competitive Price")
p.legend.location = "top_left"
show(p)


In [76]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

#Demand Price Plot
p = figure(title="Demand-Based Parking Price (Focused View)",
           x_axis_type='datetime', width=800, height=400)

p.line(price_df['Timestamp'], price_df['DemandPrice'], color='green',
       line_width=2, legend_label="Demand Price")

p.yaxis.axis_label = 'Demand Price ($)'
p.legend.location = "top_left"
show(p)


In [77]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

#Demand Price Plot
p = figure(title="Competitive-Based Parking Price (Focused View)",
           x_axis_type='datetime', width=800, height=400)

p.line(price_df['Timestamp'], price_df['CompetitivePrice'], color='red',
       line_width=2, legend_label="Competitive Price")

p.yaxis.axis_label = 'Competitive Price ($)'
p.legend.location = "top_left"
show(p)
